In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Missing').getOrCreate()

/usr/local/Cellar/jupyterlab/4.3.5/libexec/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at spark/connect/base.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/usr/local/Cellar/jupyterlab/4.3.5/libexec/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at spark/connect/commands.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/usr/local/Cellar/jupyterlab/4.3.5/libexec/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at spark/connect/common.proto. Please update the gencode 

In [3]:
spark

In [4]:
###Read dataset
training = spark.read.csv('test1.csv', header=True, inferSchema=True)

In [5]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [ ]:
['age', 'Experience'] ----> new feature ------> indeptent feature

In [8]:
from pyspark.ml.feature import VectorAssembler
featureassembler= VectorAssembler(inputCols=["age","Experience"], outputCol="Indepent Features")

In [10]:
output = featureassembler.transform(training)

In [11]:
output.show()

+---------+---+----------+------+-----------------+
|     Name|age|Experience|Salary|Indepent Features|
+---------+---+----------+------+-----------------+
|    Krish| 31|        10| 30000|      [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|       [30.0,8.0]|
|    Sunny| 29|         4| 20000|       [29.0,4.0]|
|     Paul| 24|         3| 20000|       [24.0,3.0]|
|   Harsha| 21|         1| 15000|       [21.0,1.0]|
|  Shubham| 23|         2| 18000|       [23.0,2.0]|
+---------+---+----------+------+-----------------+



In [12]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Indepent Features']

In [13]:
finalized_date = output.select("Indepent Features", "Salary")

In [14]:
finalized_date.show()

+-----------------+------+
|Indepent Features|Salary|
+-----------------+------+
|      [31.0,10.0]| 30000|
|       [30.0,8.0]| 25000|
|       [29.0,4.0]| 20000|
|       [24.0,3.0]| 20000|
|       [21.0,1.0]| 15000|
|       [23.0,2.0]| 18000|
+-----------------+------+



In [17]:
from pyspark.ml.regression import LinearRegression

##Train test spliting of data
train_data,test_data = finalized_date.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = 'Indepent Features', labelCol='Salary')
regressor= regressor.fit(train_data)

25/10/27 19:40:16 WARN Instrumentation: [46ef24e1] regParam is zero, which might cause numerical instability and overfitting.
25/10/27 19:40:17 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [18]:
###Coefficients
regressor.coefficients

DenseVector([-102.53, 1688.6818])

In [21]:
###Intercepts
regressor.intercept

16470.03994673731

In [22]:
###Prediction
pred_results=regressor.evaluate(test_data)

In [23]:
pred_results.predictions.show()

+-----------------+------+-----------------+
|Indepent Features|Salary|       prediction|
+-----------------+------+-----------------+
|       [30.0,8.0]| 25000|26903.59520639148|
+-----------------+------+-----------------+



In [27]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1903.595206391481, 3623674.709796625)

In [28]:
%config Completer.use_jedi = True